# YOLOv5 on SageMaker--Build 训练镜像

## 1 说明
本章内容为build训练镜像，推送到AWS ECR，用户可直接使用build完毕的image，不用自己build。

## 2 运行环境
Kernel 选择pytorch_latest_p36。  
本文在boto3 1.17.12和sagemaker 2.26.0下测试通过。

In [ ]:
import boto3,sagemaker
print(boto3.__version__)
print(sagemaker.__version__)

## 3 Amazon 深度学习容器

* [容器镜像清单](https://github.com/aws/deep-learning-containers/blob/master/available_images.md)
* 本文基于pytorch training: `727897471807.dkr.ecr.cn-northwest-1.amazonaws.com.cn/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04`

## 4 下载YOLOv5

In [ ]:
!git clone https://github.com/ultralytics/yolov5 container/dockersource

## 5 设置相关名称

In [ ]:
ecr_repository = 'yolov5-inference'
tag = 'latest'

## 6 Build image

In [ ]:
#国内pytorch training基础镜像地址，不要修改
base_img='727897471807.dkr.ecr.cn-northwest-1.amazonaws.com.cn/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04'
#登录基础镜像ECR，不要修改
!aws ecr get-login-password --region cn-northwest-1 | docker login --username AWS --password-stdin 727897471807.dkr.ecr.cn-northwest-1.amazonaws.com.cn

In [ ]:
%%time
%cd container
!docker build -t $ecr_repository -f Dockerfile --build-arg BASE_IMG=$base_img .
%cd ../

## 7 在本地使用容器进行训练(可选)
本地机器如果带GPU，使用`nvidia-docker run`；如果不带GPU，使用`docker run`，建议使用2xlarge以上机型，否则可能不足以分配内存。  
训练模型结果存放在`container/local_test/model/runs/train/exp/weights`

In [ ]:
!nvidia-docker run -v $(pwd)/container/local_test/:/opt/ml/ --shm-size=8g --rm $ecr_repository train
# !docker run -v $(pwd)/container/local_test/:/opt/ml/ --shm-size=8g --rm $ecr_repository train

## 8 推送到ECR

In [ ]:
!aws ecr create-repository --repository-name $ecr_repository

In [ ]:
import boto3
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
image_uri = '{}.dkr.ecr.{}.amazonaws.com.cn/{}'.format(account_id, region, ecr_repository + ":" + tag)
!docker tag $ecr_repository:$tag $image_uri
!$(aws ecr get-login --no-include-email)
!docker push $image_uri